# Librerías principales


In [2]:
import albumentations as A
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import os
import pandas as pd

# Directorios base


In [1]:
input_dir = '/home/quirogaalu/TFG/datasets/original'
output_dir = '/home/quirogaalu/TFG/datasets/augmented'
csv_path = '/home/quirogaalu/TFG/labels_regresion.csv'

conteos_path = '/home/quirogaalu/TFG/ESTADILLO_CONTEOS_FRUTA_LA_CALVILLA.xlsx'
conteos_df = pd.read_excel(conteos_path)

# Transformaciones Data Augmentation

In [4]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),                      # Flip horizontal simula cambio de orientación del árbol
    A.RandomBrightnessContrast(                   # Variaciones de luz solar y sombras
        brightness_limit=0.2,
        contrast_limit=0.2,
        p=0.5
    ),
    A.HueSaturationValue(                         # Variaciones de color por maduración o cámara
        hue_shift_limit=0,
        sat_shift_limit=10,
        val_shift_limit=10,
        p=0.3
    ),
    A.Rotate(limit=10, p=0.4),                    # Rotaciones suaves (no alterar distribución espacial drásticamente)
    A.RandomShadow(p=0.2),                        # Sombra de ramas, sol
    A.Resize(640, 640)                            # Estándar para el modelo
])

# DATA AUGMENTATION + CSV

In [5]:
from PIL import Image
import numpy as np

regression_data = []

for i, row in conteos_df.iterrows():
    arbol = row['ARBOL']
    conteo = row['NARANJAS CONTADAS']
    folder_name = f'arbol{arbol}'
    input_folder = os.path.join(input_dir, folder_name)
    output_folder = os.path.join(output_dir, folder_name)
    os.makedirs(output_folder, exist_ok=True)

    if not os.path.exists(input_folder):
        print(f'Carpeta no encontrada: {input_folder}')
        continue

    for filename in tqdm(os.listdir(input_folder), desc=f'Procesando {folder_name}'):
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Imagen original (redimensionada)
        image_resized = A.Resize(640, 640)(image=image)['image']
        save_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_orig.jpg')
        cv2.imwrite(save_path, cv2.cvtColor(image_resized, cv2.COLOR_RGB2BGR))
        regression_data.append({'ruta_imagen': save_path, 'conteo_real': conteo})

        # Generar 4 aumentaciones por imagen
        for aug_num in range(3):
            augmented = transform(image=image)['image']
            aug_filename = f'{os.path.splitext(filename)[0]}_aug{aug_num+1}.jpg'
            aug_save_path = os.path.join(output_folder, aug_filename)
            cv2.imwrite(aug_save_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))
            regression_data.append({'ruta_imagen': aug_save_path, 'conteo_real': conteo})

# Guardar CSV con etiquetas (para Regresión)
df_output = pd.DataFrame(regression_data)
df_output.to_csv(csv_path, index=False)
print(f'\n✅ CSV guardado en: {csv_path}')


Procesando arbol32: 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.12it/s]


✅ CSV guardado en: /home/quirogaalu/TFG/labels_regresion.csv
